<h1>变分推断 Variational Inference</h1>

<b>参考资料</b>

[1] An Introduction to Variational Methods for Graphical Models, Jordan etc. 1999.

[2] Variational Inference- Foundations and Modern Methods, Blei etc. NIPS Tutorial, 2016.

[3] Pattern Recognition and Machine Learning, Chapter 10, Bishop, 2006.

[4] Variational Inference: A Review for Statisticians, Blei etc. 2017.


# 0. 概率机器学习

## 0.0 概率模型

给定可观测变量$x$与隐含变量$z$， 概率模型考虑$x$与$z$的联合分布，

> $p(x, z)$

## 0.1 推断

给定观测变量，推断隐含变量$z$的后验概率分布$p(z|x)$，

> $p(z|x) = \frac{p(x, z)}{p(x)}$

给定$z$之后，$p(x, z)$的计算是比较容易的；如在GMM中知道每个样本点属于每个类的概率，则可以计算完整数据$(x, z)$的似然。

但是不完整数据$x$的似然$p(x) = \int_z p(x, z) dz$，是难以计算的；如在GMM中，需要遍历每个样本点属于每个类的概率空间。

因此需要采用一些近似推断的方式来计算$p(x)$，一般包括（1）抽样方法；（2）变分推断。本文关注<b>变分推断</b>。

# 1. 变分推断 Variational Inference

变分推断是一个确定性的近似推断方式，在有限元分析、量子力学、统计力学以及统计学等领域有着广泛的应用。

基本思想：将复杂问题（intratable）转变为简单问题，降低原始问题的自由度(degrees of freedom)。

# 1.0 ELBO (Evidence Lower BOund)

隐变量$Z$的后验分布$p(Z|X)$难以计算（intractable），用一个好计算的分布$q(Z)$来尽可能接近$p(Z|X)$。即，最小化KL散度，

> $q^*(Z) = \arg\max_{q(Z) \in \mathcal{A}} \mathbb{KL}[q(Z)||p(Z|X)] $

> $ \mathbb{KL}[q(Z)||p(Z|X)] $

> $ = \mathbb{E}_q[\log q(Z) - \log p(Z|X)]$

> $ = \mathbb{E}_q[\log q(Z) - \log p(Z, X)] + \log p(X)$

> $ = \mathbb{E}_q[\log q(Z)] - \mathbb{E}_q[\log p(Z, X)] + \log p(X)$

从而，
> $ \log p(X) = \mathbb{KL}[q(Z)||p(Z|X)] +
\underbrace{
\mathbb{E}_q[\log p(Z, X)] - \mathbb{E}_q[\log q(Z)]}_{ELBO(q)}$

> $ = \mathbb{KL}[q(Z)||p(Z|X)] + ELBO(q)$

> $ \geqslant ELBO(q)$

最小化KL散度，等价于最大化$ELBO(q)$，可以让$ELBO(q)$更接近$\log p(X)$，此时$q^*(Z) = p(Z|X)$。

> $ ELBO(q) = \mathbb{E}_q[\log p(Z, X)] - \mathbb{E}_q[\log q(Z)] $

> $ = \mathbb{E}_q[\log p(X|Z)] + \mathbb{E}_q[\log p(Z)] - \mathbb{E}_q[\log q(Z)]$

> $ = \mathbb{E}_q[\log p(X|Z)] - \mathbb{KL}[q(Z)||p(Z)] $

最大化$ELBO(q)$等价于最大化似然$\mathbb{E}_q[\log p(X|Z)]$，并且最小化$\mathbb{KL}[q(Z)||p(Z)]$，此时$q^*(Z) = p(Z)$。

# 1.1 The Mean-field Variational Family

假设隐变量之间相互独立，并且受不同的因子影响。则通用的平均场变分族为：

> $q(\boldsymbol{z}) = \prod_{j=1}^{m}q_j(z_j)$

每一个隐变量$z_j$都受它自己的变分因子$q_j(z_j)$影响。

# 1.2 Coordinate ascent mean-field variational inference

> $ ELBO(q) = \mathbb{E}_q[\log p(Z, X)] - \mathbb{E}_q[\log q(Z)] $

考虑$ELBO(q_j)$，

> $ ELBO(q_j) = \mathbb{E}_{q_j}[ \mathbb{E}_{q_{-j}} [\log p(z_j, \boldsymbol{z}_{-j}, x)] ]
- \mathbb{E}_{q_j}[\log q_j(z_j)] + const$

> $ = - \mathbb{KL}[q_j(z_j)||q^*_j(z_j)]$ 

最大化$ELBO(q_j)$等价于最小化$\mathbb{KL}[q_j(z_j)||q^*_j(z_j)]$，此时$q^*_j(z_j) = q_j(z_j)$。

坐标上升平均场变分推断CAVI(Bishop2006)，依次迭代优化平均场变分密度的每一个因子（每次迭代时，固定除该因子之外的其它因子），最终得到ELBO的局部最优解。

> $q^*_j(z_j) \propto \exp\{\mathbb{E}_{-j}[\log p(z_j|\boldsymbol{z}_{-j}, \boldsymbol{x})]\}$

> $ = \exp\{\mathbb{E}_{-j}[\log p(z_j, \boldsymbol{z}_{-j}, \boldsymbol{x}) - 
\log p(\boldsymbol{z}_{-j}, \boldsymbol{x})] \}$

> $ = \exp\{ \mathbb{E}_{-j}[\log p(z_j, \boldsymbol{z}_{-j}, \boldsymbol{x}) - 
\underbrace{
 \mathbb{E}_{-j}[\log p(\boldsymbol{z}_{-j}, \boldsymbol{x})]}_{\log p(\boldsymbol{z}_{-j}, \boldsymbol{x})} \}$

所以，
> $q^*_j(z_j) \propto \exp \{\mathbb{E}_{-j}[\log p(z_j, \boldsymbol{z}_{-j}, \boldsymbol{x})]\}$

<b>CAVI算法</b>

+ Input: 一个模型$p(X, Z)$，数据集$X$

+ Output: 一个平均场变分概率密度函数$q(\boldsymbol{z}) = \prod_{j=1}^{m}q_j(z_j)$

+ Initialize:  变分因子$q_j(z_j)$

+ while ELBO(q) 没有收敛：<br>
    for $j \in \{1,...,m\}$ <br> 
    令$q_j(z_j) \propto \exp \{\mathbb{E}_{-j}[\log p(z_j, \boldsymbol{z}_{-j}, \boldsymbol{x})]\}$ <br>
    End
    
+ 计算$ELBO(q) = \mathbb{E}_q[\log p(Z, X)] - \mathbb{E}_q[\log q(Z)]$。
    



# 1.3 CAVI示例

